This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb

cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .

In [1]:
# coding: utf-8

# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
%reload_ext autoreload
%autoreload 2

import os
import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
from utils_test import *

import cv2
from PIL import Image
import torch.nn.functional as F
from utils.utils import get_confusion_matrix
def convert_name(name):
    new = name.replace('/','_')
    return new

def get_palette(n):
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette
    
# def save_pred(preds, sv_path, name):

#     preds = preds.cpu().numpy().copy()
#     preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
#     for i in range(preds.shape[0]):
#         cv2.imwrite(os.path.join(sv_path, convert_name(name[i])) , preds[i])

        
def save_pred( preds, sv_path, name):
    palette = get_palette(256)
    preds = preds.cpu().numpy().copy()
    preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
    for i in range(preds.shape[0]):
        pred = preds[i]
        save_img = Image.fromarray(pred)
        save_img.putpalette(palette)
        save_img.save(os.path.join(sv_path, convert_name(name[i]) ))
        

def testval_lesion(config, test_dataset, testloader, model,
            sv_dir='', sv_pred=True, device = None):
    model.eval()
    confusion_matrix = np.zeros(
        (config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES))
    with torch.no_grad():
        for index, batch in enumerate(testloader):
            image, label, _, batchname = batch
            size = label.size()
            if device is None:
                image = image.cuda()
                label = label.long().cuda()
            else:
                image = image.to(device)
                label = label.long().to(device)

            pred = model(image)
            if pred.size()[-2] != size[-2] or pred.size()[-1] != size[-1]:
                pred = F.upsample(pred, (size[-2], size[-1]),
                                  mode='bilinear')

            confusion_matrix += get_confusion_matrix(
                label,
                pred,
                size,
                config.DATASET.NUM_CLASSES,
                config.TRAIN.IGNORE_LABEL)

            if sv_pred:
                sv_path = os.path.join(sv_dir, 'test_val_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                save_pred(pred, sv_path, batchname)

            if index % 100 == 0:
                logging.info('processing: %d images' % index)
                pos = confusion_matrix.sum(1)
                res = confusion_matrix.sum(0)
                tp = np.diag(confusion_matrix)
                IoU_array = (tp / np.maximum(1.0, pos + res - tp))
                mean_IoU = IoU_array.mean()
                logging.info('mIoU: %.4f' % (mean_IoU))

    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)
    pixel_acc = tp.sum() / pos.sum()
    mean_acc = (tp / np.maximum(1.0, pos)).mean()
    IoU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IoU = IoU_array.mean()

    return mean_IoU, IoU_array, pixel_acc, mean_acc

In [2]:
site_ix =17
# site_list = [2,4,6,10,11,14,17] # LN list
site_list = list(range(1,22))
cache_path = './cache/'
name = 'Cougar302_80pts_test'
# name = 'lesion_train'

cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')
print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb


from utils_test import __get_annotation__

sv_dir = mask_name = 'mask_1988'
sv_path = os.path.join(sv_dir, 'test_val_results')

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):
    if i%500==0:
        print('='*40,i)

    one = roidb[i]
    onename = one['image']
    pred_im = Image.open(os.path.join(sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox




Loading cached gt_roidb from %s ./cache/Cougar302_80pts_test_gt_roidb.pkl


/mnt/fast-data/mjc/envs/Lesions/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


======================================== 0
======================================== 500
======================================== 1000
======================================== 1500
======================================== 2000
======================================== 2500
======================================== 3000
======================================== 3500
======================================== 4000
======================================== 4500
======================================== 5000
======================================== 5500
======================================== 6000
======================================== 6500
======================================== 7000
======================================== 7500
======================================== 8000
======================================== 8500
======================================== 9000
======================================== 9500
======================================== 10000
=====================================

In [ ]:
D_CT = {}
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]

In [4]:
# In[37]:

import pandas as pd


DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/COU_baseline.csv'


if DL_info[-4:]=='.csv':
    df = pd.read_csv(DL_info)
else:
    df = pd.read_excel(DL_info)

def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        df[col] = df[col].str.replace(ori,new, case = False) 
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instence should be str or list')

pd_str_replace(df, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/W-drive")
pd_str_replace(df, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")


# In[52]:

sys.path.append('/mnt/fast-data/mjc/DeepLesion/Codes/CollectData/')
from print_weasis_raw_data import WeasisRawDataPrinter
from read_weasis_raw_data import WeasisRawFileReader


import os
import numpy as np
import pydicom
import matplotlib.pylab as plt

D_dir2header_df = {}


def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:
                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [ ]:
def compare_site(location , site_list):
    for i in site_list:
        if location == ix2labelname(i):
            return True
    return False

In [8]:
from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():
    if not compare_site(row.Location , site_list):
#         only calculate one lesion site
        continue


    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT and skip it')
        continue

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
    Metrics_vol.extend(hit)
    
    _n = len(Metrics_vol)
    if _n%100==0 or _n in [1,3,5,10,30,50]:
        print(_, _n)
        df_metrics = pd.DataFrame(Metrics_vol, 
                                  columns = ['file_name','igt','merge','#gt','#pred',
                                             'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                             'area_gt','area_pred','intersection','union']) 
        df_metrics.to_csv('temp_%s_.csv'%'ScaleNAS_Q5_9Slices_Cougar')

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
19 20
49 50
99 100


KeyError: 'ImageOrientationPatient'

In [ ]:


# site_list = [2,4,6,10,11,14,17]
# site_list = list(range(1,22))
from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():

    if _%100==0 or _<10:
        print(_)
    if not compare_site(row.Location , site_list):
#         only calculate one lesion site
        continue
    
    ## convert png numbers to dicom slice numbers.
    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT')

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=ss, gt_keep_largest=1)
    Metrics_vol.extend(hit)


In [6]:
# In[54]:

print(name)
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

Cougar302_80pts_test


,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,4.0,4.0,4.0,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,1.0,0.0,1.0,46.0,0.535551,0.661516,1.249173,0.083910,1376.750000,2985.000000,1259.000000,3102.750000
std,0.0,0.0,0.0,2.0,0.289561,0.255026,1.379038,0.033942,255.292479,1905.727333,218.661077,1878.420147
min,1.0,0.0,1.0,43.0,0.241208,0.388666,0.128327,0.033691,1052.000000,1089.000000,954.000000,1187.000000
5%,1.0,0.0,1.0,43.6,0.255039,0.405380,0.134522,0.042610,1090.100000,1186.050000,1000.200000,1295.450000
25%,1.0,0.0,1.0,46.0,0.310363,0.472236,0.159302,0.078290,1242.500000,1574.250000,1185.000000,1729.250000
50%,1.0,0.0,1.0,47.0,0.548645,0.683112,0.931119,0.097699,1411.000000,2831.500000,1311.500000,2996.000000
75%,1.0,0.0,1.0,47.0,0.773832,0.872392,2.020990,0.103320,1545.250000,4242.250000,1385.500000,4369.500000
95%,1.0,0.0,1.0,47.0,0.797732,0.887416,2.809099,0.105906,1615.450000,4998.850000,1444.300000,5059.500000
max,1.0,0.0,1.0,47.0,0.803707,0.891172,3.006126,0.106552,1633.000000,5188.000000,1459.000000,5232.000000


In [7]:
df_metrics

,file_name,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
0,/mnt/fast-disk1/mjc/AutoRecist/Pngs/COU-AA-302...,1,0,1,43,0.333415,0.500092,1.692612,0.102243,1516,3927,1361,4082
1,/mnt/fast-disk1/mjc/AutoRecist/Pngs/COU-AA-302...,1,0,1,47,0.763874,0.866132,0.169626,0.106552,1633,1736,1459,1910
2,/mnt/fast-disk1/mjc/AutoRecist/Pngs/COU-AA-302...,1,0,1,47,0.241208,0.388666,3.006126,0.033691,1306,5188,1262,5232
3,/mnt/fast-disk1/mjc/AutoRecist/Pngs/COU-AA-302...,1,0,1,47,0.803707,0.891172,0.128327,0.093156,1052,1089,954,1187


In [ ]:
stop

In [ ]:
df_metrics.to_csv('%s_metrics_vol.csv'%name)

In [9]:
SAVE_PATH = '/mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/'
name = 'Cougar_baseline_allsites'
SHOW_LABEL = True
SHOW_BOX = False
SHOW_MASK = True
SHOW_UNION_MASK= False
SHOW_MASK_LABEL = True
SHOW_GT_MASK = False

def get_proper_CT_windowing(segmentations):

    if 8 in segmentations and segmentations[8] :
        return -160 , 240
    if 7 in segmentations and segmentations[7] :
        return -160 , 240    
    if 9 in segmentations and segmentations[9] :
        return -1250 , 250
    return None

keys = list(D_CT.keys())

def compute_colors_for_labels(i):
    D = {
        1 : [1, 127, 31] , # abdomen
        2 : [64, 255, 64] , #  abdomen LN
        3 : [255, 64, 255] , # adrenal
        4 : [64, 255, 64] , # axillary LN
        5 : [5, 125, 155] , # bone
        6 : [64, 255, 64] , #  inguinal LN
        7 : [7, 124, 217] , # kidney
        8 : [8, 251, 248] , # liver
        9 : [255, 64, 64] , # lung
        10 : [64, 255, 64] , # mediastinum LN
        11 : [64, 255, 64] , # neck LN
        12 : [12, 249, 117] , # ovary
        13 : [13, 121, 148] , # pancreas
        14 : [64, 255, 64] , # pelvic LN
        15 : [15, 120, 210] , # pelvis
        16 : [16, 247, 241] , # pleural
        17 : [64, 255, 64] , #  retroperitoneal LN
        18 : [18, 246, 48] , # soft tissue
        19 : [19, 118, 79] , # spleen
        20 : [20, 245, 110] , # stomach
        21 : [21, 117, 141] , # thyroid
        }
    return D[i]

for k in keys:
    
    oneCT = remove_single_slice_segms(D_CT[k])
    savepath = os.path.join( SAVE_PATH , 'CTs_%s/%s'%(name,convert_name_compact(k)) )

    print('Image results are saving into {}'.format(savepath))

    if not os.path.exists(savepath):
        os.makedirs(savepath)
    
    for s in oneCT:
        aroidb , bboxes , segmentations = oneCT[s]

        image_path = os.path.join(aroidb['image'])
    
        CT_windowing = get_proper_CT_windowing(segmentations)
        if CT_windowing:
            HU1, HU2 = CT_windowing
        else:
            [HU1, HU2 ] = aroidb['windows']
            

        image = load_image(image_path, HU1, HU2)
        height,width = image.shape
    #     plt.imshow(image)
        image = np.dstack((image,image,image))
        
        if SHOW_MASK:
            for j in site_list:
                if j==0:
                    continue
                contours = segmentations[j]
                colors = compute_colors_for_labels(j)
                label = ix2labelname(j)
                for c in contours:
                    c = np.reshape(c,(-1,2))
                    if c.shape[0]:
                        image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
                        if SHOW_MASK_LABEL:
                            x,y,_,_ =ploy2boxes(c)
                            template = "{}"
                            if len(label)>=9:
                                label=label[:3]+label[-3:]
                
                            s = template.format(label)
                            cv2.putText(
                                image, s, (x, y), cv2.FONT_HERSHEY_SIMPLEX, .5, colors, 1
                            )  
        

        cv2.imwrite( os.path.join(savepath, convert_name_compact( aroidb['image'] ) ) , image )

Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Cougar_baseline_allsites/COU-AA-302_COU-AA-302_151983135_D2012_05_05_E4217_CT_S0601_6015
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Cougar_baseline_allsites/COU-AA-302_COU-AA-302_154670008_D2010_07_30_E1013_CT_S0002_8997
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Cougar_baseline_allsites/COU-AA-302_COU-AA-302_269580232_D2014_05_07_E3106_CT_S0002_3111
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Cougar_baseline_allsites/COU-AA-302_COU-AA-302_273330994_D2015_11_01_E3202_CT_S0002_6911
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Cougar_baseline_allsites/COU-AA-302_COU-AA-302_160040814_D2014_03_28_E6148_CT_S0015_3593
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/C